In [20]:
# Dependencies

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json

In [21]:
# Reference the file where the CSV is located
population_csv_path = "../Project1/Population_by_Zip.csv"

In [22]:
# Import the data into a Pandas DataFrame
pop_by_zip_df = pd.read_csv(population_csv_path)
pop_by_zip_df.head()

,Zip Code ZCTA,2010 Census Population
0,1001,16769
1,1002,29049
2,1003,10372
3,1005,5079
4,1007,14649


In [23]:
# Same number of values in both rows
# Both columns are made up of integers
pop_by_zip_df.count()

Zip Code ZCTA             33092
2010 Census Population    33092
dtype: int64

In [24]:
# List of Chicago Zipcodes
# 60007, 60018, 60068, 60106, 60131, 60176, 60601, 60602, 60603, 60604, 60605, 60606, 
# 60607, 60608, 60609, 60610, 60611, 60612, 60613, 60614, 60615, 60616, 60617, 60618, 
# 60619, 60620, 60621, 60622, 60623, 60624, 60625, 60626, 60628, 60629, 60630, 60631, 
# 60632, 60633, 60634, 60636, 60637, 60638, 60639, 60640, 60641, 60642, 60643, 60644, 
# 60645, 60646, 60647, 60649, 60651, 60652, 60653, 60654, 60655, 60656, 60657, 60659, 
# 60660, 60661, 60706, 60707, 60714, 60804, 60827

# Chicago Zipcodes range from 60007 - 60827

In [25]:
chicago = pop_by_zip_df.loc[pop_by_zip_df['Zip Code ZCTA'] == 60603]
chicago

,Zip Code ZCTA,2010 Census Population
20615,60603,493


In [10]:
chicago_zip = pop_by_zip_df.loc[(pop_by_zip_df['Zip Code ZCTA'] >= 60007) & (pop_by_zip_df['Zip Code ZCTA'] <= 60827)]
# chicago_zip

In [28]:
# Change column titles 
chicago_pop_df = chicago_zip.rename(columns={"Zip Code ZCTA":"Chicago Zip Code", 
                                               "2010 Census Population":"Population in 2010"})
chicago_pop_df.head(20)

,Chicago Zip Code,Population in 2010
20340,60007,33820
20341,60008,22717
20342,60010,44095
20343,60012,11120
20344,60013,26872
20345,60014,48550
20346,60015,26800
20347,60016,59690
20348,60018,30099
20349,60020,9825


In [29]:
chicago_pop_df.count()

Chicago Zip Code      337
Population in 2010    337
dtype: int64